## SDAD Broadbandnow.com Scraper Code
### Last Edit: 2023-01-12

Requires addresses from Corelogic/similar source at the block level with 2 important columns: geoid_blk and mail_address. Original approach uses one address per block group and first generates cleaned list of addresses and block groups from this. Then, proceeds to scrape all package information for those selected addresses. Reports results at block group, tract, and county level within dataframe, which can be exported.

In [2]:
# imports
# generic imports
import numpy as np
import re
import os
import time
import pandas as pd
import random
from bs4 import BeautifulSoup
from tqdm import tqdm
from slugify import slugify
import math

# selenium imports
import selenium
from selenium import webdriver
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [60]:
# check if xpath exists, if not return false
def check_exists_by_xpath(driver, xpath):
    '''
    Description:
        Check existence of xpath on page
    
    Inputs:
        webdriver: your webdriver
        xpath: whatever element we are looking for
        
    Outputs:
        returns True if xpath exists, False if not
    '''
    # try to find element
    try:
        # driver.find_element_by_xpath(xpath)
        driver.find_element("xpath", xpath)
    
    # throw exception and return false if unable to find
    except NoSuchElementException:
        return False
    return True

In [61]:
# IMPORTANT: Corelogic provides BLOCK level FIPs code, but we use BLOCK GROUP level data here
def read_and_clean_addresses_for_bgs(data, need_subset = True, size_subset = 3):
    '''
    Description:
        Check existence of xpath on page
    
    Inputs:
        data: string, name of csv you want to use (includes .csv ending)
        need_subset: boolean, True if using subset of data (originally used 1st address within each bg from list of 3) - default = True
        size_subset: integer, if subsetting, selects every "nth" row (not necessary to mess with this param if using 1 address per bg) - default = 3
        
    Outputs:
        returns True if xpath exists, False if not
    '''
    # read in csv, drop index, and update block column
    address_sample_3_per_bg = pd.read_csv(data, index_col = 0)
    address_sample_3_per_bg = address_sample_3_per_bg.reset_index(drop = True)
    address_sample_3_per_bg['geoid_blk'] = address_sample_3_per_bg.geoid_blk.astype(str)
    
    # drop lat 4 digits of mail address to get short zipcode
    a = address_sample_3_per_bg.mail_address.values
    a = np.array([a[i][0:-4] if a[i][-9].isdigit() else a[i] for i in range(len(a))])
    
    # get block group geoid
    address_sample_3_per_bg['geoid_bg'] = address_sample_3_per_bg.geoid_blk.str.slice(start=0, stop=12)
    
    # if data needs subsetting (I had 3 addresses )
    if need_subset:
        addresses = a[::size_subset]
        block_geoids = address_sample_3_per_bg.geoid_bg[::size_subset]
        
    else:
        addresses = a
        block_geoid = address_sample_3_per_bg.geoid_bg
    
    return addresses, block_geoids.values

In [62]:
# def get_empty_df(address):
#     plan_df = pd.DataFrame()
#     plan_df['address'] = address
#     plan_df['speed'] = None
#     plan_df['down_up'] = None
#     plan_df['price'] = None
#     plan_df['name'] = None
#     plan_df['type'] = None
#     plan_df['address'] = None        
#     plan_df['success'] = False
#     return plan_df

In [63]:
def search_address2(address, driver, driver_wait = 20):
    '''
    Description:
        Check existence of xpath on page
    
    Inputs:
        address: string, single home address we are scraping for
        driver: your webdriver
        driver_wait: integer, wait time for driver - default = 20
        
    Outputs:
        returns True if xpath exists, False if not
    '''
    # wait until search bar is clickable and enter address
    wait = WebDriverWait(driver, driver_wait)
    search = wait.until(EC.element_to_be_clickable((By.ID, 'plan-search')))
    search.clear()
    search.send_keys("{}".format(address))

    # sleep, then go to top suggested address
    time.sleep(sleep_time)
    go_top = check_exists_by_xpath(driver, '//*[@id="plans-search"]/div/div/div[1]/div/div/div/ul')

    # click top address
    if go_top:
        go_top_address = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="plans-search"]/div/div/div[1]/div/div/div/ul/li')))
        go_top_address.click()
        
    return go_top

def extract_page(address, driver):
    # bs - scrape page
    html = driver.page_source
    soup = BeautifulSoup(html)

    errors = soup.find_all(attrs = {"class": 'c-plans-search-error__heading'})
    
    if len(errors) > 0: # Return error address data frame if getting blocked
        return get_empty_df(address)
    
    plan_dfs = []
    # Iterate through each service plan:
    for plan in soup.find_all(attrs = {"class": 'l-providers-list__item'}):
        # Create empty data frame
        plan_df = pd.DataFrame()
        speed = [s.getText() for s in plan.find_all(attrs = {"class": "c-provider-card__speeds-value"})]
        down_up = [s.getText() for s in plan.find_all(attrs = {"class": "c-provider-card__speeds-label"})]
        price = plan.find(attrs = {"class": "c-provider-card__plan-value"}).getText().split("$")[-1]
        name = plan.find(attrs = {"class": "c-provider-card__provider-name"}).getText().split(". ")[1]
        internet_type = plan.find(attrs = {"class": "c-provider-card__label"}).getText().strip()
        
        plan_df['speed'] = speed
        plan_df['down_up'] = down_up
        plan_df['price'] = price
        plan_df['name'] = name
        plan_df['type'] = internet_type
        plan_df['address'] = address
        plan_df['success'] = True
        
        if plan_df is not None and not plan_df.empty:
            plan_dfs.append(plan_df)
            
    df = pd.concat(plan_dfs)
    # print(df)
    return df

In [64]:
def scrape_prices(driver, addresses, min_wait = 10, max_wait =30, save_folder ='../../data/temp/'):
    '''
    Description:
        Scrape internet packages from Broadbandnow.com - takes each address and scrapes all packages for top match
    
    Inputs:
        driver: your webdriver
        addresses: array of strings, home addresses we are scraping for (first output of read_and_clean_addresses_for_bgs)
        
    Outputs:
        df: a data frame containing the columns:
            address: corresponding address of the package
            price: price of the package
            name: name of the package
            type: type of package?
            speed: speed of the package
            
    '''
    adfs = []
    empty = []
    
    # loop over block group addressed
    for address in tqdm(addresses):
        # try below and exception IF takes too long (increments a counter before skipping address eventually)
        
        address_name = slugify(address)
        if os.path.isfile(save_folder + address_name +'.csv'):            
            continue
            
        try:
            # reload page to clear results (noticed that we run into issues if we do not clear)
            driver.get("https://broadbandnow.com/compare/plans")
            go_top = search_address2(address, driver)

            # select top address
            if not go_top:
                # print('Skipping: Cannot go to the top address')
                empty.append(address)
                continue # skip to next address
            time.sleep(1)
            unable_to_confirm = check_exists_by_xpath(driver, "/html/body/div[2]/div/div/div[1]/section/section/div/div/div[1]/div/section")

            # if able to confirm and go to top axrddress
            if unable_to_confirm:
                # print('Skipping: Cannot go to the top address')
                empty.append(address)
                continue
                
            time.sleep(1)
            load_more = check_exists_by_xpath(driver, '//*[@id="cityPlansListing"]/section/div/div[2]/div/div/section')

            #if load more is an option, then load all packages
            if load_more:
                # load all plans
                load_all_plans = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="cityPlansListing"]/section/div/div[2]/div/div/section')))
                load_all_plans.click()

            adf = extract_page(address, driver)

            if not adf.empty:
                # adfs.append(adf)
                adf.to_csv(save_folder + address_name +'.csv', index=False)
            
            # Be respectful of pinging the server
            time.sleep(random.randint(min_wait, max_wait))
            
            # select edit option to change address
            edit = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="plans-search"]/div/div/div/h1/span')))
            edit.click()

        # if try fails, throw exception and increment counter (retry until problem_counter hits 5)
        # throws error if we try to edit search plans but this is not an option because nothing was searched after hitting home page
        except TimeoutException as ex:
            # DO something
            empty.append(address)

    # close driver
    driver.quit()        

    return empty


In [65]:
# Storing previous column names
#     df = pd.DataFrame({"address": result, "price": flatten(all_prices), "name": flatten(all_names),
#                        "type": flatten(all_type_list), "download": download, "upload": upload,
#                        "block_group": short_blockgroup_geoid2, "tract": short_tract_geoid2,
#                        "county": short_county_geoid2})

### Running the functions above

In [66]:
# start driver
options = Options()
# Uncomment to run headless (i.e., don't show the browser)
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')  # Last I checked this was necessary.
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

driver.get("https://broadbandnow.com/compare/plans")



#driver.maximize_window()

# set driver params
driver_wait = 5
sleep_time = 2
wait = WebDriverWait(driver, driver_wait)

/var/folders/31/_flrh_251y578sg72lx1yb780000gn/T/ipykernel_60282/3253202446.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
/var/folders/31/_flrh_251y578sg72lx1yb780000gn/T/ipykernel_60282/3253202446.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [6]:
# df = pd.read_csv('../../data/reverse_county_addresses.csv')
df = pd.read_csv('../../data/01073_one_address_per_block.csv')
df

,address,geoid20,lon,lat,geometry,centroid
0,"9256 PARKWAY E, BIRMINGHAM, AL, 35206",10730001001000,-86.700589,33.587030,"POLYGON ((-86.702001 33.587957, -86.701786 33....",POINT (-86.69980215252554 33.589449975959404)
1,"9002 PARKWAY E, BIRMINGHAM, AL, 35206",10730001001002,-86.716150,33.577447,"POLYGON ((-86.718615 33.577959, -86.716288 33....",POINT (-86.70931800982687 33.585290442879064)
2,"9001 ROEBUCK BLVD, BIRMINGHAM, AL, 35206",10730001001003,-86.708240,33.583977,"POLYGON ((-86.711173 33.581161, -86.711101 33....",POINT (-86.70888940717963 33.5829721121745)
3,"116 94TH ST N, BIRMINGHAM, AL, 35206",10730001001004,-86.715152,33.588869,"POLYGON ((-86.719281 33.581638, -86.719147 33....",POINT (-86.71476793686115 33.58580244498006)
4,"101 94TH ST N, BIRMINGHAM, AL, 35206",10730001001005,-86.714927,33.588854,"POLYGON ((-86.716521 33.588634, -86.716439 33....",POINT (-86.71563553503205 33.58860715803149)
...,...,...,...,...,...,...
15310,"5234 COTTAGE LN, BIRMINGHAM, AL, 35226",10730144152012,-86.869407,33.362234,"POLYGON ((-86.869641 33.361759, -86.869569 33....",POINT (-86.869311135407 33.3621089765314)
15311,"5302 COTTAGE LN, BIRMINGHAM, AL, 35226",10730144152015,-86.870224,33.360721,"POLYGON ((-86.87211 33.356841, -86.872061 33.3...",POINT (-86.87085499910671 33.35876491281303)
15312,"129 COBBLESTONE LN, BIRMINGHAM, AL, 35244",10730144152017,-86.873603,33.349815,"POLYGON ((-86.87791 33.348207, -86.877876 33.3...",POINT (-86.8742913777893 33.348216662686994)
15313,"124 COBBLESTONE LN, BIRMINGHAM, AL, 35244",10730144152018,-86.873539,33.349793,"POLYGON ((-86.873893 33.350597, -86.873852 33....",POINT (-86.87332028866619 33.35043097691216)


In [7]:
# sample = df.sample(3)[['address', 'geoid', 'name']]
# addresses = list(sample['address_found'])
addresses = list(df['address'])
print(len(addresses))

addresses = [v for v in addresses if isinstance(v, str)]
print(len(addresses))

# block_geoids = np.array(df['geoid'])
# print(len(block_geoids))

15315
15315


#  Probably better to save the files individually before adding together to save progress (and memory)

In [9]:
print(len(os.listdir('../../data/temp/')))

0


In [10]:
# iterate only through ones that do not exist
# print(slugify(addresses[0]))
# print(os.listdir('../../data/temp/')[0])

addresses = [a for a in addresses if not slugify(a)+'.csv' in os.listdir('../../data/temp/')]
print(len(addresses))

15315


In [72]:
empty = scrape_prices(driver, addresses, 3, 5)
print('Empty: %s' % empty)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 10228/10228 [33:30:37<00:00, 11.79s/it]


Empty: ['Tekoa Mountain Wildlife Management Area, Main Street, Russell, Hampden County, Massachusetts, 01071, United States', 'Cool Coastal Cartz, 8428, 20th Street, Vero Beach, Indian River County, Florida, 32966, United States', '150th Avenue, Pocahontas County, Iowa, 50554, United States', 'Oswichee, Russell County, Alabama, 36856, United States', 'Amazon DEW5, Princess Road, Franklin Corner, Lawrence Township, Mercer County, New Jersey, 08648, United States', 'Lakewood Drive Southeast, Spring Creek Ridge, Altoona, Polk County, Iowa, 50009, United States', 'Paris Bottoms Road, Logan County, Arkansas, United States', 'I 37, Hilltop Acres Colonia, Edroy, San Patricio County, Texas, 78368, United States', 'East Mitchell Road, Five Points, Union County, Indiana, 47353, United States', 'County Road 742, Woodruff County, Arkansas, United States', 'US 70, Haleys Grove, Crab Orchard, Cumberland County, East Tennessee, Tennessee, 38555, United States', 'Mariposa County, CAL Fire Southern Reg

# Save found empty addresses to a list

In [74]:
len(empty)

5231

In [75]:
with open('../../data/broadband_empty.txt', 'w') as f:
    for line in empty:
        f.write(f"{line}\n")

# Combine and append geoid based on address

In [3]:
d ='../../data/temp/'
l= []
for file in tqdm(os.listdir(d)):
    if file.split('.')[-1] == 'csv':
        filepath = os.path.join(d, file)
        df = pd.read_csv(filepath)
        l.append(df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6723/6723 [00:04<00:00, 1345.18it/s]


In [17]:
cdf = pd.concat(l)
cdf

,speed,down_up,price,name,type,address,success
0,5000 Mbps,Download,180.00,AT&T Fiber,Internet 5000,"5332, Carol Plantation Road, Quail Run, Mobile...",True
1,5000 Mbps,Upload,180.00,AT&T Fiber,Internet 5000,"5332, Carol Plantation Road, Quail Run, Mobile...",True
2,1200 Mbps,Download,149.99,Xfinity,Super+ More,"5332, Carol Plantation Road, Quail Run, Mobile...",True
3,940 Mbps,Download,70.00,CenturyLink Fiber Gigabit,NaN,"5332, Carol Plantation Road, Quail Run, Mobile...",True
4,940 Mbps,Upload,70.00,CenturyLink Fiber Gigabit,NaN,"5332, Carol Plantation Road, Quail Run, Mobile...",True
...,...,...,...,...,...,...,...
93,3 Mbps,Upload,109.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True
94,25 Mbps,Download,139.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True
95,3 Mbps,Upload,139.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True
96,25 Mbps,Download,159.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True


In [34]:
cdf.iloc[[1]]['address'].values[0].split(',')[-4]

' Mobile County'

In [39]:
cdf['check'] = cdf['address'].apply(lambda x: len(x.split(',')))

In [42]:
cdf['check'].unique()

array([ 7,  6,  8,  4,  5,  3,  9,  2, 10])

In [43]:
cdf[cdf['check'] <= 2]

,speed,down_up,price,name,type,address,success,check,check
0,1000 Mbps,Download,99.99,Cox Communications,Internet Gigablast,"New Mexico, United States",True,2,2
1,35 Mbps,Upload,99.99,Cox Communications,Internet Gigablast,"New Mexico, United States",True,2,2
2,1000 Mbps,Download,197.99,Cox Communications,Internet Gigablast + Contour TV Preferred,"New Mexico, United States",True,2,2
3,35 Mbps,Upload,197.99,Cox Communications,Internet Gigablast + Contour TV Preferred,"New Mexico, United States",True,2,2
4,1000 Mbps,Download,217.99,Cox Communications,Internet Gigablast + Contour TV Preferred + Vo...,"New Mexico, United States",True,2,2
...,...,...,...,...,...,...,...,...,...
104,3 Mbps,Upload,109.99,HughesNet,Internet Only,"Puerto Rico, United States",True,2,2
105,25 Mbps,Download,139.99,HughesNet,Internet Only,"Puerto Rico, United States",True,2,2
106,3 Mbps,Upload,139.99,HughesNet,Internet Only,"Puerto Rico, United States",True,2,2
107,25 Mbps,Download,159.99,HughesNet,Internet Only,"Puerto Rico, United States",True,2,2


In [46]:
cdf[cdf['check'] == 7].iloc[[1]]['address'].values[0].split(',')

['5332',
 ' Carol Plantation Road',
 ' Quail Run',
 ' Mobile County',
 ' Alabama',
 ' 36582',
 ' United States']

# Because there might be inconsistencies? Iterate through each and append to a list before adding the geodi

In [52]:
geoid_col = []
for i in tqdm(range(len(cdf))):
    address = cdf.iloc[[i]]['address'].values[0]
    pdf = df[df['address'] == address]
    # print(pdf)
    if not pdf.empty:
        geoid_col.append(pdf['geoid'].values[0])
    else:
        geoid_col.append(None)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 483596/483596 [04:23<00:00, 1838.12it/s]


In [55]:
cdf['geoid'] = [str(v) for v in geoid_col]

In [56]:
cdf

,speed,down_up,price,name,type,address,success,check,check,geoid
0,5000 Mbps,Download,180.00,AT&T Fiber,Internet 5000,"5332, Carol Plantation Road, Quail Run, Mobile...",True,7,7,1097
1,5000 Mbps,Upload,180.00,AT&T Fiber,Internet 5000,"5332, Carol Plantation Road, Quail Run, Mobile...",True,7,7,1097
2,1200 Mbps,Download,149.99,Xfinity,Super+ More,"5332, Carol Plantation Road, Quail Run, Mobile...",True,7,7,1097
3,940 Mbps,Download,70.00,CenturyLink Fiber Gigabit,NaN,"5332, Carol Plantation Road, Quail Run, Mobile...",True,7,7,1097
4,940 Mbps,Upload,70.00,CenturyLink Fiber Gigabit,NaN,"5332, Carol Plantation Road, Quail Run, Mobile...",True,7,7,1097
...,...,...,...,...,...,...,...,...,...,...
93,3 Mbps,Upload,109.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True,3,3,29155
94,25 Mbps,Download,139.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True,3,3,29155
95,3 Mbps,Upload,139.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True,3,3,29155
96,25 Mbps,Download,159.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True,3,3,29155


In [59]:
cdf['unit'] = cdf['speed'].apply(lambda x: x.split(' ')[-1])

In [60]:
cdf['speed'] = cdf['speed'].apply(lambda x: x.split(' ')[0])

In [61]:
cdf

,speed,down_up,price,name,type,address,success,check,check,geoid,unit
0,5000,Download,180.00,AT&T Fiber,Internet 5000,"5332, Carol Plantation Road, Quail Run, Mobile...",True,7,7,1097,Mbps
1,5000,Upload,180.00,AT&T Fiber,Internet 5000,"5332, Carol Plantation Road, Quail Run, Mobile...",True,7,7,1097,Mbps
2,1200,Download,149.99,Xfinity,Super+ More,"5332, Carol Plantation Road, Quail Run, Mobile...",True,7,7,1097,Mbps
3,940,Download,70.00,CenturyLink Fiber Gigabit,NaN,"5332, Carol Plantation Road, Quail Run, Mobile...",True,7,7,1097,Mbps
4,940,Upload,70.00,CenturyLink Fiber Gigabit,NaN,"5332, Carol Plantation Road, Quail Run, Mobile...",True,7,7,1097,Mbps
...,...,...,...,...,...,...,...,...,...,...,...
93,3,Upload,109.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True,3,3,29155,Mbps
94,25,Download,139.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True,3,3,29155,Mbps
95,3,Upload,139.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True,3,3,29155,Mbps
96,25,Download,159.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",True,3,3,29155,Mbps


In [62]:
cdf = cdf[['speed','down_up','price','name','type','address','geoid','unit']]
cdf

,speed,down_up,price,name,type,address,geoid,unit
0,5000,Download,180.00,AT&T Fiber,Internet 5000,"5332, Carol Plantation Road, Quail Run, Mobile...",1097,Mbps
1,5000,Upload,180.00,AT&T Fiber,Internet 5000,"5332, Carol Plantation Road, Quail Run, Mobile...",1097,Mbps
2,1200,Download,149.99,Xfinity,Super+ More,"5332, Carol Plantation Road, Quail Run, Mobile...",1097,Mbps
3,940,Download,70.00,CenturyLink Fiber Gigabit,NaN,"5332, Carol Plantation Road, Quail Run, Mobile...",1097,Mbps
4,940,Upload,70.00,CenturyLink Fiber Gigabit,NaN,"5332, Carol Plantation Road, Quail Run, Mobile...",1097,Mbps
...,...,...,...,...,...,...,...,...
93,3,Upload,109.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",29155,Mbps
94,25,Download,139.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",29155,Mbps
95,3,Upload,139.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",29155,Mbps
96,25,Download,159.99,HughesNet,Internet Only,"Pemiscot County, Missouri, United States",29155,Mbps


In [64]:
cdf.to_csv('../../data/usa_county_broadbandnow_pirces.csv.xz', index=False)